# NYC Yellow Taxi Rides

## I. Overview

### 1. Abstract

Using **data science** techniques and the open dataset from NYC's TLC, we've managed to discover some interesting insights in the way the Yellow Taxicabs operated in the first half of the year 2019.

### 2. Keywords

nyc, taxi, yellow-taxi, tlc, data-engineering, data-analysis

### 3. Introduction

NYC is popular for many things. One of these things is the yellow taxicab. They are widely recognizable symbols of the city. Taxis painted yellow (medallion taxis) are able to pick up passengers anywhere in the five boroughs. Taxicabs are operated by private companies and licensed by the New York City Taxi and Limousine Commission (TLC).

Luckily for us, [TLC Trip Record Data](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page) is an open dataset which we can use to gain some interesting insights on how people in NYC use them.

We are going to use a Data Science workflow to achieve our goals.

**Data Acquisition --> Data Exploration --> Data Pre-processing --> Analysis**


### 4. Project Objectives

The objective of this project is to apply *data science* techniques to discover useful insights in the data provided by NYC's TLC (Taxi & Limousine Commision) for the period January 2019 - June 2019.

## II. The Data

### 1. Source
[TLC Trip Record Data](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

We are going to use the trip record data starting from January 2019 up to June 2019. We are also going to use the Taxi Zone Lookup Table also provided by this source.

### 2. Description
**A. TRIPS DATA**
- **Files**: yellow_tripdata_2019-[01-06].csv (header included)
- **Summary**: Each record models a yellow taxi trip.

|**Column**|**Description**|
|---|---|
|VendorID|A code indicating the TPEP provider that provided the record. **<ul><li>1= Creative Mobile Technologies, LLC;</li> <li>2= VeriFone Inc. </li></ul>**|
|tpep_pickup_datetime|The date and time when the meter was engaged.|
|tpep_dropoff_datetime|The date and time when the meter was disengaged.|
|passenger_count|The number of passengers in the vehicle. This is a driver-entered value.|
|trip_distance|The elapsed trip distance in miles reported by the taximeter.|
|RatecodeID|The final rate code in effect at the end of the trip. **<ul><li>1=Standard rate</li> <li>2=JFK</li> <li>3=Newark</li> <li>4=Nassau or Westchester</li><li>5=Negotiated fare</li> <li>6=Group ride</li></ul>**|
|store_and_fwd_flag|This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server. **<ul><li>Y=store and forward trip</li><li>N=not a store and forward trip</li></ul>**|
|PULocationID|TLC Taxi Zone in which the taximeter was engaged|
|DOLocationID|TLC Taxi Zone in which the taximeter was disengaged|
|payment_type|A numeric code signifying how the passenger paid for the trip. **<ul><li>1=Credit card</li><li>2=Cash</li><li>3=No charge</li><li>4=Dispute</li><li>5=Unknown</li><li>6=Voided trip</li></ul>**|
|fare_amount|The time-and-distance fare calculated by the meter.|
|extra|Miscellaneous extras and surcharges.  Currently, this only includesthe 0.50 and 1 rush hour and overnight charges.|
|mta_tax|0.50 MTA tax that is automatically triggered based on the metered rate in use.|
|tip_amount|This field is automatically populated for credit card tips. Cash tips are not included.|
|tolls_amount|Total amount of all tolls paid in trip.|
|improvement_surcharge|0.30 improvement surcharge assessed trips at the flag drop.|
|total_amount|The total amount charged to passengers. Does not include cash tips.|
|congestion_surcharge|Surcharge in the case of congestion.|

- **Details**:
For further details please refer to the [trip record user guide](https://www1.nyc.gov/assets/tlc/downloads/pdf/trip_record_user_guide.pdf) and the [yellow trips data dictionary](https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf), both provided by TLC.

**B. TAXI ZONE LOOKUP TABLE**
- **Files**: taxi+_zone_lookup.csv (header included)
- **Summary**: Each record models a taxi zone.

|**Column**|**Description**|
|---|---|
|"LocationID"|ID of the location|
|"Borough"|Borough in which the taxi zone is located|
|"Zone"|Name of the zone|
|"service_zone"|Type of zone, irrelevant for the yellow taxicabs as these are not restricted|

- **Details**: For further details please refer to the [trip record user guide](https://www1.nyc.gov/assets/tlc/downloads/pdf/trip_record_user_guide.pdf) provided by TLC. Maps of the taxi zones for each borough can be found [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

## III. The Environment

On the current environment, only spark is installed so we need to also install the other tools.

In [1]:
%pip install matplotlib
%pip install seaborn

     |████████████████████████████████| 11.6 MB 343 kB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 3.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 283 kB 752 kB/s eta 0:00:01
     |████████████████████████████████| 10.5 MB 229 kB/s eta 0:00:01
     |████████████████████████████████| 25.9 MB 13.1 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 5.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
%%bash
apt-get install --assume-yes wget

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libpcre2-8-0
The following NEW packages will be installed:
  libpcre2-8-0 wget
0 upgraded, 2 newly installed, 0 to remove and 0 not upgraded.
Need to get 1115 kB of archives.
After this operation, 3925 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian buster/main amd64 libpcre2-8-0 amd64 10.32-5 [213 kB]
Get:2 http://deb.debian.org/debian buster/main amd64 wget amd64 1.20.1-1.1 [902 kB]
Fetched 1115 kB in 1s (1969 kB/s)
Selecting previously unselected package libpcre2-8-0:amd64.
(Reading database ... 27735 files and directories currently installed.)
Preparing to unpack .../libpcre2-8-0_10.32-5_amd64.deb ...
Unpacking libpcre2-8-0:amd64 (10.32-5) ...
Selecting previously unselected package wget.
Preparing to unpack .../wget_1.20.1-1.1_amd64.deb ...
Unpacking wget (1.20.1-1.1) ...
Setting up libpcre2-8-0:amd64 (10.32-5) ...
Se

debconf: delaying package configuration, since apt-utils is not installed


## IV. Data Acquisition

First we need to download the data. We are going to simply use wget to download the necessary csv files in a new directory called *nyc-taxi-data*.

This might take a while as the total size of the data is almost 4GB. It is recommended not to run the next cell if you already have the data.

In [5]:
%%bash
mkdir nyc-taxi-data
wget -nv -P nyc-taxi-data https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-01.csv
wget -nv -P nyc-taxi-data https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-02.csv
wget -nv -P nyc-taxi-data https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-03.csv
wget -nv -P nyc-taxi-data https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-04.csv
wget -nv -P nyc-taxi-data https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-05.csv
wget -nv -P nyc-taxi-data https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-06.csv
wget -nv -P nyc-taxi-data https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

mkdir: cannot create directory ‘nyc-taxi-data’: File exists
2020-09-09 14:49:15 URL:https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-01.csv [687088084/687088084] -> "nyc-taxi-data/yellow_tripdata_2019-01.csv" [1]
2020-09-09 15:03:47 URL:https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-02.csv [649882828/649882828] -> "nyc-taxi-data/yellow_tripdata_2019-02.csv" [1]
2020-09-09 15:19:47 URL:https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-03.csv [726201566/726201566] -> "nyc-taxi-data/yellow_tripdata_2019-03.csv" [1]
2020-09-09 15:30:28 URL:https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-04.csv [689207122/689207122] -> "nyc-taxi-data/yellow_tripdata_2019-04.csv" [1]
2020-09-09 15:43:15 URL:https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-05.csv [701538890/701538890] -> "nyc-taxi-data/yellow_tripdata_2019-05.csv" [1]
2020-09-09 15:53:22 URL:https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-06.csv [6434

## V. Data Exploration

After we acquired the data, we need to get familiar with it, identify missing values, trends, outliers and check the veridicity of the documentation. This can help us understand the data better so that we know what alterations are needed in the next phase of Data Pre-Processing.

Please note that there are other data exploration objectives like finding correlations between variabes. However, in this chapter we focus mainly on techniques that help us understand how we can clean and improve the data that we have.

Before anything else, let's import the packages that we need and create the connection to the spark cluster.

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.\
        builder.\
        appName("nyc-taxi-app").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1g").\
        getOrCreate()

First let's read the trips data from all 6 csv trip files in a single DataFrame. 

In [10]:
trips = spark.read.csv('nyc-taxi-data/yellow_tripdata_2019-*.csv', header=True, inferSchema=True)

First, let's have a look at the metadata.

In [11]:
print(f'Columns: {len(trips.columns)}')

Columns: 18


In [12]:
print(f'Rows: {trips.count()}')

Rows: 44459136


In [13]:
trips.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



So it looks like there were **44.5 MILION** yellow taxi trips in NYC in the first 6 months of 2019.

We can already see that the columns **extra**, **mta_tax**,**tolls_amount** and **improvement_surcharge** can be dropped later in the pre-processing stage as they do not provide much information for us and won't be useful for our purpose. 

Let's make use of the describe() function and identify if we have missing values and where they are.

In [16]:
trips.describe().show()

+-------+------------------+--------------------+---------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-------------------+------------------+------------------+---------------------+------------------+--------------------+
|summary|          VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|   passenger_count|    trip_distance|        RatecodeID|store_and_fwd_flag|      PULocationID|     DOLocationID|       payment_type|       fare_amount|             extra|            mta_tax|        tip_amount|      tolls_amount|improvement_surcharge|      total_amount|congestion_surcharge|
+-------+------------------+--------------------+---------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-------------------+---------------

We can see that the only column with missing values seem to be **congestion_surcharge** which was to be expected as not all trips suffer congestion. However, the number of values in that column is still high (only 5M rows seem to have missing values) so we should expect to see many 0 values in that column.

However, the **min** and **max** summaries give us more valuable information. There seem to be rows with corrupted values, e.g. datetimes in 2001 and 2088, negative or unrealistic amounts, invalid RatecodeIDs etc. These outliers need to be removed.

Let's test some of our assumptions to see if they are true so that we know how to proceed next.
First, let's check if there is a considerable amount of 0.0 in the congestion_surcharge column.

In [19]:
trips.filter('congestion_surcharge = 0.0').count()

6222601

It looks like only 6.2M rows have 0.0 congestion surcharge so our assumption wasn't true. It probably is common to have this surcharge in a nyc taxicab. So in order not to drop 5M rows which have this missing values and do not make our assumption that it was 0.0, we should fill the missing values with the mean for this column.

Now, we know from the documentation provided by TLC that there are only 2 possible VendorID values, 1 and 2. However, before considering any different value an outlier, let's see weather or not there are other legitimate VendorIDs which were simply not documented.

In [22]:
trips.groupBy('VendorID').count().collect()

[Row(VendorID=1, count=16603925),
 Row(VendorID=4, count=256291),
 Row(VendorID=2, count=27598920)]

Looks like there is another Vendor with the ID of 4. Even if it accounts for only less than 0.01% of the data, we will still keep these observations as it is unrealistic to assume that 250000 rows are corrupted only because the VendorID differs.

Now, let's do the same with the RatecodeID.

In [23]:
trips.groupBy('RatecodeID').count().collect()

[Row(RatecodeID=1, count=42936836),
 Row(RatecodeID=6, count=288),
 Row(RatecodeID=3, count=90733),
 Row(RatecodeID=5, count=275287),
 Row(RatecodeID=4, count=32722),
 Row(RatecodeID=2, count=1121694),
 Row(RatecodeID=99, count=1576)]

There seems to be another RatecodeID=99. Since we don't know what this means and the number of rows with this RatecodeID is fairly low, we can simply modify these observations in the Pre-Processing stage to Unknown=5.

Finally, let's check how many rows we will need to drop because they have negative or unrealistic amount values. 

In [26]:
trips.filter('fare_amount < 0.0 or fare_amount > 100000.0 or tip_amount < 0.0 or total_amount < 0.0 or total_amount > 200000.0').count()

67972

## VI. Data Pre-Processing

**Assumptions**:
- There is another undocumented vendor with the VendorID of 4 which reported ~250000 trips.
- The missing values in the congestion_surcharge are simply missing values.
- The RatecodeID=99 is Unknown.
- The fare_amount is in the range [0.0, 100000] and the total_amount in the range [0.0, 200000]. The rest of the values are corrupted / outliers.

**Clean & Transform Steps**:
1. Drop extra, mta_tax,tolls_amount and improvement_surcharge columns;
2. Switch type of datetime columns to timestamp
3. Remove outlier rows: datetimes outside Jan - Jun 2019 | negative or unrealistcly high amounts | 0 trip_distance or passenger_count
4. Fill missing values for congestion_surcharge with mean (assume there was 0.0 congestion surcharge in these cases)
5. Modify RatecodeID=99 to 5 (Unknown)
6. Integrate with the taxi zone lookup table to add 2 new columns: PUBorough and DOBorough
7. Add a new column based on pickup_datetime: day_of_week (integer numbers from 1=Monday to 7=Sunday)

In [ ]:
# 1. Drop extra, mta_tax,tolls_amount and improvement_surcharge columns;

In [ ]:
# 2. Switch type of datetime columns to timestamp

In [ ]:
# 3. Remove outlier rows: datetimes outside Jan - Jun 2019 | negative or unrealistcly high amounts | 0 trip_distance or passenger_count

In [ ]:
# 4. Fill missing values for congestion_surcharge with mean (assume there was 0.0 congestion surcharge in these cases)

In [ ]:
# 5. Modify RatecodeID=99 to 5 (Unknown)

In [ ]:
# 6. Integrate with the taxi zone lookup table to add 2 new columns: PUBorough and DOBorough

In [ ]:
# 7. Add a new column based on pickup_datetime: day_of_week (integer numbers from 1=Monday to 7=Sunday)